# Integración de Cadenas en Proyectos : Runnable, OutputParser y Streaming

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 913.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [19]:
from langchain_core.runnables import RunnableLambda

sequence = RunnableLambda(lambda x: x +1 ) | RunnableLambda(lambda x: x * 2 )

In [20]:
sequence.invoke(10)

22

In [21]:
sequence = RunnableLambda(lambda x: x +1 ) | {
    'index_1' : RunnableLambda(lambda x: x * 2 ),
    'index_2' : RunnableLambda(lambda x: x * 5 )
}

In [22]:
sequence.invoke(10)

{'index_1': 22, 'index_2': 55}

In [6]:
!pip install -qU langchain langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 7.9 MB/s eta 0:00:00


In [7]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [8]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [9]:
model = ChatOpenAI()

In [10]:
joke_query = 'Tell me a joke'

In [11]:
parser = JsonOutputParser()

In [12]:
prompt = PromptTemplate(
    template = 'Answer the user query. \n{format_instructions}\n{query}',
    input_variables = ['query'],
    partial_variables = { 'format_instructions': parser.get_format_instructions()}
)

In [13]:
chain = prompt | model | parser

In [14]:
response = chain.invoke({"query": joke_query})

In [15]:
import time
for s in chain.stream({"query": joke_query}):
  print(s)
  time.sleep(0.3)

{}
{'joke': ''}
{'joke': 'Why'}
{'joke': 'Why couldn'}
{'joke': "Why couldn't"}
{'joke': "Why couldn't the"}
{'joke': "Why couldn't the bicycle"}
{'joke': "Why couldn't the bicycle stand"}
{'joke': "Why couldn't the bicycle stand up"}
{'joke': "Why couldn't the bicycle stand up by"}
{'joke': "Why couldn't the bicycle stand up by itself"}
{'joke': "Why couldn't the bicycle stand up by itself?"}
{'joke': "Why couldn't the bicycle stand up by itself? Because"}
{'joke': "Why couldn't the bicycle stand up by itself? Because it"}
{'joke': "Why couldn't the bicycle stand up by itself? Because it was"}
{'joke': "Why couldn't the bicycle stand up by itself? Because it was two"}
{'joke': "Why couldn't the bicycle stand up by itself? Because it was two tired"}
{'joke': "Why couldn't the bicycle stand up by itself? Because it was two tired!"}


In [23]:
chunks = []
async for chunk in model.astream(joke_query):
  chunks.append(chunk)
  print(chunk.content, end='', flush=True)
  time.sleep(0.3)

Why couldn't the bicycle stand up by itself?

Because it was two-tired!